<center><h1>Classification by regional modeling</h1></center>

Classification by regional modeling consists in a five-step approach:
1. Setting the hyper-parameters. In this step, we specify the number of SOM prototypes $C$. It must be also defined as
the maximum number of regions $K_{max}$. Without any prior knowledge, we will set in this example $K_{max} = \sqrt{C}$.


2. SOM training. In order to build regional models, follow the procedure introduced by Vesanto and Alhoniemi [1].
Thus, the very first step requires training the SOM as usual, with $C$ prototypes.


3. Clustering of the SOM. The step consists in performing clustering over the $C$ SOM prototypes. Although one may
use any clustering algorithm for this step, for the sake of simplicity, we use the standard K-means algorithm in
combination with the Davies–Bouldin (DB) index. The DB index is a clustering validity measure commonly used for
finding the optimal number of clusters, but any suitable measure can be equally used (see [2]). Thus, we compute
$K = 1, 2, ... K_{max}$ partitioning of the SOM prototypes and the corresponding DB index value as well.
The optimal partitioning, represented by $K_{opt}$ partitions, is then the value of $K$ wich minimizes the DB index.


4. Partitioning SOM prototypes into regions. Once $K_{opt}$ is selected, the $r$-th cluster of SOM prototypes,
$r = 1...K_{opt}$, is composed of all weight vectors $w_i$ that are mapped onto the prototype $p_r$ of the K-means
algorithm. More formally, the set of SOM prototypes associated with the r-th prototype of the K-means algorithm
is defined as: $$W_r = \{w_i \in R^{p+q} | \|w_i-p_r\| < \|w_i-p_j\|, \forall j =1,...,K_{opt}, j\neq r \}$$


5. Mapping data points to regions. The fourth step consists in finding $K_{opt}$ data partitions, denoted by
$\{X_1\}$, $\{X_2\}$, ... , $\{X_{K_{opt}}\}$ of the training dataset by mapping each datapoint to a region
$r \in \{1, ... , K_{opt}\}$. In other words, let us denote $N_r$ as the number of data vectors in $\{X_r\}$.
Then, the partition $\{X_r\}$ is composed of those input vectors $x_{rμ}$, $μ = 1, ... , N_r$ , whose closest SOM
prototype belongs to $W_r$.


6. Building classification models over the regions. Finally, once the original dataset has been divided into $K_{opt}$
subsets (one per region), the last step consists in building $K_{opt}$ regional classification models using
$X_r$, $r = 1, ... , K_{opt}$.

* Vertebral Column
* Wall-Following
* Alzheimer


### References

[1] J. Vesanto, E. Alhoniemi, Clustering of the self-organizing map, IEEE Trans.
Neural Netw. 11 (2000) 586–600.

[2] M. Halkidi, Y. Batistakis, M. Vazirgiannis, On clustering validation techniques, J. Intell. Inf. Syst. 17 (2001) 107–145.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%%time

import numpy as np 
import pandas as pd 

from math import ceil
from load_dataset import datasets

from devcode.utils import dummie2multilabel, scale_feat
from devcode.models.som import SOM
from devcode.models.regional_learning import RegionalModel
from devcode import run_round

from devcode.utils.metrics import DB

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.cluster import KMeans


dataset_name='pk'

X = datasets[dataset_name]['features'].values
Y = datasets[dataset_name]['labels'].values

test_size = 0.2

# Train/Test split = 80%/20%
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=test_size)

# scaling features
X_tr_norm, X_ts_norm = scale_feat(X_train, X_test, scaleType='min-max')

#N = len(dataset['features'].index) # number of datapoints
N = len(X_tr_norm) # number of datapoints in the train split
l = ceil((5*N**.5)**.5) # side length of square grid of neurons

som = SOM(l,l)
som_params={
    'alpha0':    0.01,
    'sigma0':    1,
    'nEpochs':   1,
    'verboses':  0            
}

C = l**2 # number of SOM neurons in the 2D grid
k_values = [i for i in range(2, ceil(np.sqrt(C)))] # 2 to sqrt(C)
cluster_params={
    'n_clusters': {'metric':   DB,        # when a dictionary is pass a search begins
                   'criteria': np.argmin, # search for smallest DB score 
                   'k_values': k_values}, # around the values provided in 'k_values'
    'n_init':     10, # number of initializations
    'init':       'random', 
    #'n_jobs':     -1
}

linearModel = linear_model.LinearRegression(n_jobs=-1)

rm = RegionalModel(som, linearModel)
rm.fit(X=X_tr_norm, Y=y_train, verboses=0,
        SOM_params     = som_params,
        Cluster_params = cluster_params)

# Evaluating in the test dataset
y_pred = rm.predict(X_ts_norm)
y_pred = np.round(np.clip(y_pred, 0, 1)) # rounding prediction numbers

cm = confusion_matrix(dummie2multilabel(y_test),
                      dummie2multilabel(y_pred))
#cm = np.asarray(cm).reshape(-1) # matrix => array
acc=0
total=sum(sum(cm))
for j in range(len(cm)):
    acc += cm[j,j] # summing the diagonal
acc/=total

Dataset:  Features.shape:   # of classes:
vc2c      (310, 6)          2
vc3c      (310, 6)          3
wf24f     (5456, 24)        4
wf4f      (5456, 4)         4
wf2f      (5456, 2)         4
pk        (195, 22)         2


CPU times: total: 1.56 s
Wall time: 1.25 s


In [3]:
acc

0.8461538461538461

In [4]:
run_round(X, Y, test_size, RegionalModel, {"SOM_class": som, "Model_class": linearModel}, is_rounded=True)

AttributeError: 'NoneType' object has no attribute 'labels_'

Evaluation of regional OLS in the datasets:

In [ ]:
# constant hyperparameters:
test_size = 0.2
scaleType = 'min-max'
n_resamplings = 100

# hyperparameters grid search:
num = 3
alphas = np.linspace(0.1, 0.5,  num=num).tolist()
sigmas = np.linspace(3,    10,   num=num).tolist()
epochs = np.linspace(100,  500, num=num, dtype='int').tolist()

# vector of random states for train/test split
random_states = np.random.randint(np.iinfo(np.int32).max, size=n_resamplings).tolist()
cases = [
    {
         "dataset_name" : dataset_name
        ,"random_state":  random_state
        ,"som_params":    { "alpha0"  : alpha0
                           ,"sigma0"  : sigma0
                           ,"nEpochs" : nEpochs
                          }
    }
    # hyperparameters possible values
    for dataset_name in datasets.keys()
    for random_state in random_states
    for alpha0       in alphas
    for sigma0       in sigmas
    for nEpochs      in epochs
]

print("alphas: {}\nsigmas: {}\nepochs: {}\n".format(alphas,sigmas,epochs))

print("# of alphas: {}\n# of sigmas: {}\n# of epochs: {}\n# of random_states: {}\n# of datasets: {}\n".format(
    len(alphas), len(sigmas), len(epochs), len(random_states), len(list(datasets.keys()))))

print("# of cases: {}".format(len(cases)))

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from functools import partial
from multiprocessing import Pool

def evalRLM(case):
    dataset_name = case['dataset_name']
    random_state = case['random_state']
    som_params   = case['som_params']
    
    X = datasets[dataset_name]['features'].values
    Y = datasets[dataset_name]['labels'].values
    
    # Train/Test split
    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=test_size, random_state=random_state)
    # scaling features
    X_tr_norm, X_ts_norm = scale_feat(X_train, X_test, scaleType=scaleType)

    N = len(X_tr_norm) # number of datapoints in the train split
    l = ceil((5*N**.5)**.5) # side length of square grid of neurons

    som = SOM(l,l)

    C = l**2 # number of SOM neurons in the 2D grid
    k_values = [i for i in range(2, ceil(np.sqrt(C)))] # 2 to sqrt(C)
    cluster_params={
        'n_clusters': {'metric':   DB,        # when a dictionary is pass a search begins
                       'criteria': np.argmin, # search for smallest DB score 
                       'k_values': k_values}, # around the values provided in 'k_values'
        'n_init':     10, # number of initializations
        'init':       'random'
        #'n_jobs':     0
    }

    linearModel = linear_model.LinearRegression()

    rlm = RegionalModel(som, linearModel)
    rlm.fit(X=X_tr_norm, Y=y_train,
            SOM_params     = som_params,
            Cluster_params = cluster_params)

    # Evaluating in the train set
    y_tr_pred = rlm.predict(X_tr_norm)
    y_tr_pred = np.round(np.clip(y_tr_pred, 0, 1)) # rounding prediction numbers

    cm_tr = confusion_matrix(dummie2multilabel(y_train),
                             dummie2multilabel(y_tr_pred)
                            ).reshape(-1) # matrix => array

    # Evaluating in the test set
    y_ts_pred = rlm.predict(X_ts_norm)
    y_ts_pred = np.round(np.clip(y_ts_pred, 0, 1)) # rounding prediction numbers

    cm_ts = confusion_matrix(dummie2multilabel(y_test),
                             dummie2multilabel(y_ts_pred)
                            ).reshape(-1) # matrix => array

    data = [dataset_name, random_state]+list(som_params.values())+[cm_tr]+[cm_ts]
    return data

In [5]:
# browser notification when cell finishs with '%%notify'
# import jupyternotify
# ip = get_ipython()
# ip.register_magics(jupyternotify.JupyterNotifyMagics)

In [6]:
%%notify
from multiprocessing import Pool
import tqdm

data = [None]*len(cases)
count=0
pool = Pool()
for i in tqdm.tqdm(pool.imap_unordered(evalRLM, cases), total=len(cases)):
    data[count] = i
    count+=1
pool.close()
pool.join()


results = np.vstack(data)
header  = ["dataset_name", "random_state", "alpha0", "sigma0", "nEpochs", "cm_tr", "cm_ts"]
results_df = pd.DataFrame(results, columns=header)

filename = "ROLS - all - n_res={n_resamplings} - {datetime}.csv".format(
    n_resamplings=n_resamplings,
    datetime=datetime.datetime.now()
)
results_df.to_csv(filename,index=False) # saving results in csv file

# [{elapsed}<{remaining}

UsageError: Cell magic `%%notify` not found.


### Processing results:

In [7]:
# loading simulation results
df_results = pd.read_csv('ROLS - all - n_res=100 - 2019-07-10 04:50:57.404253.csv')
df_results.head()

OSError: [Errno 22] Invalid argument: 'ROLS - all - n_res=100 - 2019-07-10 04:50:57.404253.csv'

Results for each data set:

In [8]:
som_params = [
    {
     "alpha0"  : alpha0
    ,"sigma0"  : sigma0
    ,"nEpochs" : nEpochs
    }
    for alpha0       in alphas
    for sigma0       in sigmas
    for nEpochs      in epochs
]

header = list(som_params[0].keys()) + ['Minimum', 'Maximum', 'Median', 'Mean', 'Std. Deviation']

df_ds = {}
for dataset_name in datasets: # For this specific dataset
    print(dataset_name)
    df = df_results.loc[df_results['dataset_name'] == dataset_name] # get simulation results

    count = 0
    df_data   = np.zeros((len(som_params), len(header))) # matriz que guardará resultados numéricos
    for params in som_params:
        df_case = df.loc[(df['alpha0']  == params['alpha0']) & 
                         (df['sigma0']  == params['sigma0']) &
                         (df['nEpochs'] == params['nEpochs'])]

        # converting confusion matrix from string to numpy array
        cm_ts = np.array([[int(x) for x in result[1:-1].split()] for result in df_case['cm_ts'].values])

        #data = cm_ts
        length = cm_ts.shape[1]
        cm_side = int(np.sqrt(length))

        acc   = [0]*len(cm_ts)
        for i in range(len(cm_ts)):
            cm = np.reshape(cm_ts[i], (cm_side,cm_side))
            acc[i] = np.trace(cm)/np.sum(cm)

        df_data[count,:] = np.matrix([
            params['alpha0'], params['sigma0'], params['nEpochs'],
            min(acc), max(acc), np.median(acc), np.mean(acc), np.std(acc)])
        count+=1


    df_ds[dataset_name] = pd.DataFrame(df_data, columns=header)
    print(df_ds[dataset_name].head()) # TODO: display
    print('-'*100,'\n'*2)

NameError: name 'alphas' is not defined

Taking the best values by higher mean in accuracy.

In [9]:
data = np.array([df.sort_values('Mean', ascending=False).iloc[0,:].values for df in df_ds.values()])
idx_label = list(df_ds.keys())
df_rols = pd.DataFrame(data, columns=header, index=[idx_label])
df_rols

NameError: name 'df_ds' is not defined

# Globlal OLS

In [10]:
# constant hyperparameters:
test_size = 0.2
scaleType = 'min-max'
n_resamplings = 100

# vector of random states for train/test split
random_states = np.random.randint(np.iinfo(np.int32).max, size=n_resamplings).tolist()
cases = [
    {
         "dataset_name" : dataset_name
        ,"random_state":  random_state
    }
    for dataset_name in datasets.keys()
    for random_state in random_states
]

print("# of random_states: {}\n# of datasets: {}\n".format(
    len(random_states), len(list(datasets.keys()))))

print("# of cases: {}".format(len(cases)))

# of random_states: 100
# of datasets: 6

# of cases: 600


In [11]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import linear_model

def evalGOLS(case):
    dataset_name = case['dataset_name']
    random_state = case['random_state']
    
    X = datasets[dataset_name]['features'].values
    Y = datasets[dataset_name]['labels'].values
    
    # train/test split
    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=test_size, random_state=random_state)

    # scaling features
    X_tr_norm, X_ts_norm = scale_feat(X_train, X_test, scaleType='min-max')

    model = linear_model.LinearRegression().fit(X_tr_norm,y_train)
    
    # Evaluating in the train set
    y_tr_pred = model.predict(X_tr_norm)
    y_tr_pred = np.round(np.clip(y_tr_pred, 0, 1)) # rounding prediction numbers

    cm_tr = confusion_matrix(dummie2multilabel(y_train),
                             dummie2multilabel(y_tr_pred)
                            ).flatten() # matrix => array
    
    # Evaluating in the test set
    y_ts_pred = model.predict(X_ts_norm)
    y_ts_pred = np.round(np.clip(y_ts_pred, 0, 1)) # rounding prediction numbers

    cm_ts = confusion_matrix(dummie2multilabel(y_test),
                             dummie2multilabel(y_ts_pred)
                            ).flatten() # matrix => array

    
    data = [dataset_name, random_state]+[cm_tr]+[cm_ts]
    return data

In [ ]:
from multiprocessing import Pool
import tqdm

data = [None]*len(cases)

pool = Pool()
data =[result for result in tqdm.tqdm(pool.imap_unordered(evalGOLS,cases), total=len(cases))]
pool.close()
pool.join()

results = np.vstack(data)
header  = ["dataset_name", "random_state", "cm_tr", "cm_ts"]
results_df = pd.DataFrame(results, columns=header)

  0%|                                                                                                                               | 0/600 [00:00<?, ?it/s]

In [ ]:
results_df.head()

Processing results (taking the best values by higher mean in accuracy):

In [ ]:
header = ['Minimum', 'Maximum', 'Median', 'Mean', 'Std. Deviation']

data      = np.zeros(( len(datasets.keys()), len(header) ))
idx_label = [' ']*len(datasets.keys())
count=0
for dataset_name in datasets: # For this specific dataset
    df = results_df.loc[results_df['dataset_name'] == dataset_name] # get simulation results
    
    # converting confusion matrices to numpy matrix
    cm_ts = np.array([array for array in df['cm_ts'].values])
       
    length = cm_ts.shape[1]
    cm_side = int(np.sqrt(length))
    acc   = [0]*len(cm_ts)
    for i in range(len(cm_ts)):
        cm = np.reshape(cm_ts[i], (cm_side,cm_side))
        acc[i] = np.trace(cm)/np.sum(cm)

    data[count,:] = np.array([min(acc), max(acc), np.median(acc), np.mean(acc), np.std(acc)])
    idx_label[count] = dataset_name
    count+=1
    
df_ols = pd.DataFrame(data, columns=header, index=[idx_label])
df_ols

# Comparing results:

In [ ]:
header = ['Dataset', 'Model']+list(df_ols.columns)

temp_rols = df_rols.rename_axis('Dataset').reset_index().loc[:,[x for x in header if x!='Model']]
temp_rols.insert(1,'Model',['ROLS']*len(datasets.keys()))

temp_ols = df_ols.rename_axis('Dataset').reset_index()
temp_ols.insert(1,'Model',['OLS']*len(datasets.keys()))

print(
    pd.concat([temp_ols,temp_rols]).sort_index()
) # TODO: display